In [6]:
import sys
sys.path.insert(1, 'src')

from src.coins import *
from src.generator import generate_random_model
from src.reverse_engineer import reverse_engineer_model
from src.error import calculate_model_error_all


In [ ]:
NUM_COINS = 1000
NUM_FLIPS = 1e4
SIZE_RANGE = (1, 5)
MEMORY_DEPTH_RANGE = (1, 4)

models = []
memory_depth_data = []
complexity_data = {}
error_data = {}

DEBUG = True

for i in range(NUM_COINS):
    input_model = generate_random_model(size_range=SIZE_RANGE, memory_depth_range=MEMORY_DEPTH_RANGE, benchmark=True, benchmark_flips=int(NUM_FLIPS))
    output_models = reverse_engineer_model(input_model.benchmark_result.flip_history, input_model, benchmark=True, benchmark_flips=int(NUM_FLIPS))
    errors = calculate_model_error_all(input_model, output_models)

    if DEBUG:
        print(f'Iteration {i+1}: n={input_model.size}, m={input_model.memory_depth}, error={np.round(errors, 1)}')

    if not (input_model.memory_depth in complexity_data):
        complexity_data[input_model.memory_depth] = []
        error_data[input_model.memory_depth] = []

    models.append(input_model)
    memory_depth_data.append(input_model.memory_depth)
    complexity_data[input_model.memory_depth].append(input_model.complexity)
    error_data[input_model.memory_depth].append(np.array(errors).transpose())

memory_depth_data = set(memory_depth_data)
min_memory_depth = min(memory_depth_data)
max_memory_depth = max(memory_depth_data)


In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import colors as mcolors

plt.figure()
ax = plt.gca()  # Get the current axis

plt.xlabel('Complexity')
plt.ylabel('Relative Error')
plt.title(f'Relative Error vs. Complexity ({NUM_COINS} Systems, {NUM_FLIPS:.0E} Flips)')

# Define colormap
cmap = plt.cm.viridis
norm = mcolors.BoundaryNorm(boundaries=np.arange(min_memory_depth - 0.5, max_memory_depth + 1.5), 
                            ncolors=cmap.N)

# Plot with color-coded memory depth
for i in complexity_data.keys():
    ax.scatter(complexity_data[i], error_data[i], color=cmap(norm(i)), alpha=0.5)

# Create ScalarMappable for colorbar
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])  # Fixes the error

# Add colorbar to the current axis
plt.colorbar(sm, ax=ax, 
             ticks=range(min_memory_depth, max_memory_depth+1), 
             label="Model Memory Depth")

plt.show()
